# Convolutional Neural Networks
이번 시간에는 간단하게 CNN을 구현해 보도록 하겠습니다. 이번 시간 역시 mnist를 가지고 하겠습니다.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline


mnist = input_data.read_data_sets("./mnist", one_hot=True)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

print "x_train: ", x_train.shape
print "y_train: ", y_train.shape
print "x_test: ", x_test.shape
print "y_test: ", y_test.shape

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
x_train:  (55000, 784)
y_train:  (55000, 10)
x_test:  (10000, 784)
y_test:  (10000, 10)


### Parameter setting 

이제 CNN에 들어갈 각종 parameter들을 설정합니다. 통상적으로 CNN에는 다음과 같은 parameter들이 필요합니다.
- learning rate
- batch size
  * CNN을 학습할 때 가장 좋은 방법은 N개의 샘플이 있는 데이터를 한꺼번에 input으로 넣어서 학습하는 건데, training set이 50000개 이렇게 되면 메모리 문제가 발생할 수 있습니다. 그래서 할수없이 데이터를 가령 500개씩 분할(minibatch)로 나눠서 학습을 하게 됩니다. 이렇게 되면 50000개를 한번에 학습하는 대신, 500개만을 넣은 모델을 학습하고, 같은 모델을 다음 500개의 데이터로 다시 학습하는 걸 100번 하는 식으로 진행이 됩니다.
- training iterations 혹은 number of epochs
- input size 
  * 통상적으로 이미지를 행렬로 나타낼 때 [H(높이),W(너비),D(RGB)]로 나타냅니다. MNIST는 28x28의 흑백이미지이므로 input size는 28x28x1=784가 되고, CIFAR-10의 경우 32x32x10, imageNet은 224x224x3 등으로 해상도에 따라 달라집니다.
- number of classes

In [2]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 100
display_step = 10

In [3]:
# Network parameters
n_input = 784
n_classes = 10
dropout = 0.5

Input & Output, 그리고 사용되는 dropout값을 담기 위한 placeholder를 지정합니다.

In [4]:
# Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # to keep dropout probability

CNN 내부에 사용되는 filter(=layer)(=weight) 및 bias들을 Variable의 형태로 생성합니다.
- 얘네가 굳이 Variable인 이유는 CNN을 학습할 때 backpropagation을 계산하면서 값들을 매번 업데이트해줘야 하기 때문입니다.
- weight랑 bias를 생성할 때, 내가 원하는 CNN이 어떻게 convolution 혹은 pooling을 하면서 input과 output matrix의 크기가 어떻게 달라지는지를 매번 계산해야 합니다. 마지막에 fully connected layer를 만들 때에는 계산이 복잡해질 수 있는데, 뒤에 이를 좀 더 쉽게 하는 법이 나옵니다.

In [ ]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

** Network Implementation **

이제 CNN architecture를 한번 구현해 보겠습니다. 우리 CNN은 다음과 같은 간단한 구조를 가집니다.

1) 5x5 convolution (x32)

2) ReLU

3) Max pooling

4) 5x5 convolution (x64)

5) ReLU

6) Max pooling

7) Fully connected layer

이 모든 과정은 TF library 의 함수 6개만 알면 됩니다.

1) tf.nn.conv2d(input, weights, strides, padding)
- 용도: 원본 layer 부분부분마다 필터를 대고 2-D convolution을 진행해서 output layer를 만든다.
* input: 원본 이미지의 placeholder 혹은 전 단계의 결과
* weights: 아까 tf.Variable로 만들어놓은, 해당 단계에 써야 하는 weight
* strides: [1,int,int,1]이나 대개 [1,1,1,1]로 합니다.
* padding: 'SAME' 혹은 'VALID'가 있는데 거의 다 'SAME' 씁니다. ('SAME'이 zero padding 허용)

2) tf.nn.max_pool(input, ksize, strides, padding)
- 용도: window를 이미지에다 대고 해당 window 내 가장 큰 값만 반환하면서 축소된 layer를 만든다.
* input: 원본 이미지의 placeholder 혹은 전 단계의 결과
* ksize: [1,int,int,1], 여기서 int에는 1/int로 배율을 줄일 지를 나타냅니다.
* strides: [1,int,int,1]이나 대개 ksize랑 똑같이 맞춥니다.
* padding: 역시 'SAME'으로 맞춥시다. ('SAME'이 zero padding 허용)

3) tf.nn.bias_add(input, bias_layer)
- 용도: bias를 더해준다 (Wx+b의 b를 더해주는 겁니다)
* input: 그냥 인풋
* bias_layer: 앞서 weight와 함께 정의한 해당 bias입니다.

4) tf.nn.relu(input)
- 용도: nonlinearity를 만들어주기 위해, 가장 많이 쓰이는 ReLU 함수를 원본 layer에다 적용합니다.
* input: 역시 그냥 인풋

In [ ]:
def CNN(x, weights, biases, dropout):
    
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    print "Input x:               ", x.get_shape().as_list()

    # 1st Convolution Layer
    conv1 = tf.nn.conv2d(x, weights['wc1'], strides=[1,1,1,1], padding='SAME')
    print "After 1st conv:        ", conv1.get_shape().as_list()
    conv1 = tf.nn.bias_add(conv1, biases['bc1'])
    print "After adding bias:     ", conv1.get_shape().as_list()
    conv1 = tf.nn.relu(conv1)
    print "After ReLU:            ", conv1.get_shape().as_list()
    # Max Pooling (down-sampling)
    pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    print "After 1st max_pooling: ", pool1.get_shape().as_list()

    # 2nd Convolution Layer
    conv2 = tf.nn.conv2d(pool1, weights['wc2'], strides=[1,1,1,1], padding='SAME')
    print "After 2nd conv:        ", conv2.get_shape().as_list()
    conv2 = tf.nn.bias_add(conv2, biases['bc2'])
    print "After adding bias:     ", conv2.get_shape().as_list()
    conv2 = tf.nn.relu(conv2)
    print "After ReLU:            ", conv2.get_shape().as_list()
    # Max Pooling (down-sampling)
    pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    print "After 2nd max_pooling: ", pool2.get_shape().as_list()

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(pool2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    print "Fully connected:       ", fc1.get_shape().as_list()
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    print "Output:                ", out.get_shape().as_list()
    return out

** Optimization **

이제 CNN 모델의 구조를 지었으니, 실제로 결과를 출력하게끔 해야죠.
위의 def CNN이 결과물로 [None,10]을 뱉어냈는데, 이제 loss를 계산하고 optimize를 해야죠.
구조는 다음과 같습니다.

1) pred : 각 input을 넣었을 때 10개의 class에 대한 값들이 저장되어 있다
  - a: [100,10]라고 하자
  - a[0,0]=0.18, a[0,1]=0.02, a[0,2]=0.95, ..., a[0,9]=0.37 이런 식이면 a[0]이라는 샘플은 [2]에 해당하는 label, 즉 '1'일 거라고 예측
  
2) 예측한 pred와 실제 y를 받아서 softmax 계산을 하고 loss를 반환하는 cost. y 역시 [100,10]여야 한다.

3) 그 cost를 인풋으로 받고, cost를 최대한으로 줄이기 위한 함수를 실행하는 optimizer를 생성한다.
- optimizer들 종류는 다음과 같다. 아니, 더 많다.

In [ ]:
# Construct model
pred = CNN(x, weights, biases, keep_prob)
# Define loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
# Define optimizer
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
# 다른 optimizer들도 한번 넣어보세요
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
#optimizer = tf.train.AdadeltaOptimizer().minimize(cost)
#optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate).minimize(cost)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.5).minimize(cost)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

** Performance metric **

다음 함수들을 이용해서 accuracy를 측정할 수 있습니다.

In [ ]:
# Find correct prediction
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
# Get accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# initialize variables
init = tf.initialize_all_variables()

연구실에서 서버 돌릴 때 꿀팁, GPU 사용 관련 커맨드 알려드립니다

1) 사용방법: tf.Session 혹은 tf.InteractiveSession의 argument로 해당 config 추가

2) tf.ConfigProto 커맨드
  - log_device_placement=True : 어느 device를 사용하는지를 알기 위한 device mapping입니다.
  - allow_soft_placement=True : device를 설정했는데 해당 device가 없어서 (혹은 메모리가 다 차서) 안 돌아갈 
    수 있는데, 이 커맨드를 통해 그런 문제를 방지할 수 있습니다.
  - 제 경우에는 allow_soft_placement=True로 하니까, GPU로 지원되지 않는 단순 연산도 GPU가 하려고 하다 보니 발생하는 에러를 줄일 수 있었습니다. 사용범위를 유기적으로 조정하지 않을까 생각합니다.

  ### tf.GPUOptions 커맨드
      - (1) ** allow_growth=True ** : Tensorflow에서는 디폴트로 사용가능한 모든 메모리를 할당하는데, 이렇게 되면 mnist 하나 돌린다고 11기가씩 잡아먹을 수 있습니다. 이를 방지하기 위해 allow_growth=True로 하면 필요한 만큼의 메모리만 먹습니다.
        - 11749MiB->461MiB
      - (2) ** per_process_gpu_memory_fraction=0.1 ** : 역시 이 커맨드를 켜면 GPU 최대 메모리 중 0.1, 즉 10%만을
      사용한다는 뜻입니다. 돌려야 하는 네트워크가 allow_growth로도 많은 메모리를 잡아먹게 되면 이 커맨드를
      통해 강제할당할 수 있습니다. 필요에 맞게 퍼센트를 조절할 수 있습니다.

In [ ]:
gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.1)
config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True,
                        gpu_options=gpu_options)

In [ ]:
# apply config when starting session
sess = tf.InteractiveSession(config=config)
init.run()
step = 1

while step * batch_size < training_iters:
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    # Run optimization
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
    if step % display_step == 0:
    # Calculate batch loss and accuracy
        loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
        print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
    step += 1
print("Optimization Finished!")

In [ ]:
    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

편의를 위해 weight, bias를 따로 설정하지 않는 방법도 있습니다. 지난 시간 때 배운 tf.get_variable()을 이용하면 됩니다.

In [5]:
def conv2d_simple(x, filter_size, dim_in, dim_out, name, strides=1):
    with tf.variable_scope(name) as scope:
        w = tf.get_variable('w', shape=[filter_size, filter_size, dim_in, dim_out],
                initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
        b = tf.get_variable('b', shape=[dim_out])
        x = tf.nn.conv2d(x,w,strides=[1,strides,strides,1],padding='SAME')
        x = tf.nn.bias_add(x,b)
        return tf.nn.relu(x)

In [6]:
def fc_simple(x,dim_fc,n_classes,name):
    with tf.variable_scope(name) as scope:
        size_arr = x.get_shape().as_list()
        print type(size_arr)
        print size_arr[1]
        dim_in = np.prod(np.asarray(size_arr[1:]))
        print dim_in
        print x.get_shape().as_list()
        x = tf.reshape(x, [-1,dim_in])
        dim_in = x.get_shape().as_list()[1]
        print dim_in
        wd = tf.get_variable('wd', shape=[dim_in, dim_fc],
                initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
        bd = tf.get_variable('bd', shape=[dim_fc])
        wout = tf.get_variable('wout', shape=[dim_fc, n_classes],
                initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
        bout = tf.get_variable('bout', shape=[n_classes])
        fc1 = tf.add(tf.matmul(x, wd),bd)
        fc1 = tf.nn.relu(fc1)
        fc1 = tf.nn.dropout(fc1,dropout)
        
        out = tf.add(tf.matmul(fc1, wout),bout)
        return out 

In [7]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')

In [8]:
def CNN_simple(x, dropout):
    
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    print "x shape", x.get_shape().as_list
    # 1st Convolution Layer
    conv1 = conv2d_simple(x, 5, 1, 32, 'conv1')
    print "After 1st conv:        ", conv1.get_shape().as_list()
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv1, k=2)
    print "After 1st max_pooling: ", conv1.get_shape().as_list()

    # 2nd Convolution Layer
    conv3 = conv2d_simple(conv2, 5, 32, 64, 'conv2')
    print "After 2nd conv:        ", conv2.get_shape().as_list()
    # Max Pooling (down-sampling)
    conv4 = maxpool2d(conv3, k=2)
    print "After 2nd max_pooling: ", conv2.get_shape().as_list()

    # Fully connected layer
    out = fc_simple(conv4, 1024, n_classes, 'fc1')
    return out

In [ ]:
def VGGNet_simple(x, dropout):
    
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 224, 224, 3])
    print "x shape", x.get_shape().as_list
    conv1 = conv2d_simple(x, 3, 3, 64, 'conv1')
    print "After 1st conv:        ", conv1.get_shape().as_list()
    conv2 = conv2d_simple(x, 3, 64, 64, 'conv2')
    print "After 2nd conv:        ", conv1.get_shape().as_list()
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv1, k=2)
    print "After 1st max_pooling: ", conv1.get_shape().as_list()

    # 2nd Convolution Layer
    conv3 = conv2d_simple(conv2, 5, 32, 64, 'conv2')
    print "After 2nd conv:        ", conv2.get_shape().as_list()
    # Max Pooling (down-sampling)
    conv4 = maxpool2d(conv3, k=2)
    print "After 2nd max_pooling: ", conv2.get_shape().as_list()

    # Fully connected layer
    out = fc_simple(conv4, 1024, n_classes, 'fc1')
    return out

In [9]:
#tf.get_variable_scope().reuse_variables()
# Construct model
pred = CNN_simple(x, keep_prob)
# Define loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
# Define optimizer
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
# 다른 optimizer들도 한번 트라이 해보세요
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
#optimizer = tf.train.AdadeltaOptimizer().minimize(cost)
#optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate).minimize(cost)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.5).minimize(cost)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

x shape <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(1)])>
After 1st conv:         [None, 28, 28, 32]
After 1st max_pooling:  [None, 28, 28, 32]
After 2nd conv:         [None, 14, 14, 32]
After 2nd max_pooling:  [None, 14, 14, 32]
<type 'list'>
7
3136
[None, 7, 7, 64]
3136


In [10]:
# Find correct prediction
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
# Get accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# initialize variables
init = tf.initialize_all_variables()

In [11]:
gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.333)
config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True,
                        gpu_options=gpu_options)

In [12]:
# apply config when starting session
sess = tf.InteractiveSession(config=config)
init.run()
step = 1

while step * batch_size < training_iters:
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    # Run optimization
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
    if step % display_step == 0:
    # Calculate batch loss and accuracy
        loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
        print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
    step += 1
print("Optimization Finished!")

Iter 1000, Minibatch Loss= 5.156737, Training Accuracy= 0.16000
Iter 2000, Minibatch Loss= 3.788680, Training Accuracy= 0.07000
Iter 3000, Minibatch Loss= 2.893919, Training Accuracy= 0.14000
Iter 4000, Minibatch Loss= 2.974441, Training Accuracy= 0.16000
Iter 5000, Minibatch Loss= 2.865923, Training Accuracy= 0.11000
Iter 6000, Minibatch Loss= 2.589398, Training Accuracy= 0.16000
Iter 7000, Minibatch Loss= 2.277774, Training Accuracy= 0.18000
Iter 8000, Minibatch Loss= 2.523856, Training Accuracy= 0.25000
Iter 9000, Minibatch Loss= 1.936950, Training Accuracy= 0.33000
Iter 10000, Minibatch Loss= 1.393814, Training Accuracy= 0.52000
Iter 11000, Minibatch Loss= 0.833924, Training Accuracy= 0.75000
Iter 12000, Minibatch Loss= 0.840184, Training Accuracy= 0.73000
Iter 13000, Minibatch Loss= 0.434115, Training Accuracy= 0.85000
Iter 14000, Minibatch Loss= 0.328565, Training Accuracy= 0.85000
Iter 15000, Minibatch Loss= 0.401292, Training Accuracy= 0.86000
Iter 16000, Minibatch Loss= 0.5365

In [13]:
    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

('Testing Accuracy:', 0.9765625)
